# Hacker News articles analysis

You can find the data set [here](https://www.kaggle.com/hacker-news/hacker-news-posts), but note that we have reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that didn't receive any comments and then randomly sampling from the remaining submissions. You can download this downsampled data [here](https://dq-content.s3.amazonaws.com/356/hacker_news.csv).


Below are descriptions of the columns:

* **id**: the unique identifier from Hacker News for the post
* **title**: the title of the post
* **url**: the URL that the posts links to, if the post has a URL
* **num_points**: the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* **num_comments**: the number of comments on the post
* **author**: the username of the person who submitted the post
* **created_at**: the date and time of the post's submission

## Setup

In [8]:
# Imports
from csv import reader
import codecs
import datetime as dt

In [2]:
def open_file(filename, encoding="utf-8"):
    opened_file = codecs.open(filename, "r", encoding)
    read_file = reader(opened_file)
    apps_data = list(read_file)

    return apps_data

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [3]:
hacker_news_data = open_file('data/hacker_news.csv')

hacker_news_data_header = hacker_news_data[0]
hacker_news_data_content = hacker_news_data[1:]

In [4]:
print("Hacker News data header:\n", hacker_news_data_header)

explore_data(hacker_news_data_content, 0, 5, rows_and_columns=True)

Hacker News data header:
 ['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/

## Task statement

We're specifically interested in posts with titles that begin with either **Ask HN** or **Show HN**. Users submit **Ask HN** posts to ask the Hacker News community a specific question. Below are a few examples:

* Ask HN: How to improve my personal website?
* Ask HN: Am I the only one outraged by Twitter shutting down share counts?
* Ask HN: Aby recent changes to CSS that broke mobile?


Likewise, users submit **Show HN** posts to show the Hacker News community a project, product, or just something interesting. Below are a few examples:

* Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform'
* Show HN: Something pointless I made
* Show HN: Shanhu.io, a programming playground powered by e8vm


We'll compare these two types of posts to determine the following:

1. Do Ask HN or Show HN receive more comments on average?
2. Do posts created at a certain time receive more comments on average?

## Extracting Ask and Show post separately

In [5]:
ask_posts = []
show_posts = []
other_posts = []

for row in hacker_news_data_content:
    if len(row) != 7:
        continue
    title = row[1]
    if title.lower().startswith('ask hn'):
        ask_posts.append(row)
    elif title.lower().startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)

print("Number of 'Ask' posts:", len(ask_posts))
print("Number of 'Show' posts:", len(show_posts))
print("Number of other posts:", len(other_posts))

Number of 'Ask' posts: 1744
Number of 'Show' posts: 1162
Number of other posts: 17192


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

In [6]:
def find_average_number_of_comments_in_dataset(dataset):
    total_comments = 0

    for row in dataset:
        number_of_comments = int(row[4])
        total_comments += number_of_comments
    
    avg_comments = total_comments/len(dataset)
    return avg_comments

avg_ask_comments = find_average_number_of_comments_in_dataset(ask_posts)
print("Average number of comments to 'Ask' posts:", avg_ask_comments)
avg_show_comments = find_average_number_of_comments_in_dataset(show_posts)
print("Average number of comments to 'Show' posts:", avg_show_comments)
avg_other_comments = find_average_number_of_comments_in_dataset(other_posts)
print("Average number of comments to other posts:", avg_other_comments)

Average number of comments to 'Ask' posts: 14.038417431192661
Average number of comments to 'Show' posts: 10.31669535283993
Average number of comments to other posts: 26.87598883201489


## Finding the Number of Ask Posts and Comments by Hour Created

We'll focus our remaining analysis just on 'Ask' posts.

Next, we'll determine if 'Ask' posts created at a certain *time* are more likely to attract comments.

### Calculate the amount of ask posts created during each hour of day and the number of comments received.

In [9]:
date_format = "%m/%d/%Y %H:%M"

for row in ask_posts:
    created_at = row[6]
    try:
        row[6] = dt.datetime.strptime(created_at, date_format)
    except ValueError:
        row[6] = created_at  # If the format is incorrect, leave the value as string
        print(f'created_at for the row {row} was incorrect, returning it as string')

In [18]:
created_at_hours = {}
comments_at_hours = {}

for row in ask_posts:
    created_at = row[6]
    amount_of_comments = int(row[4])
    created_at_time = created_at.time()
    created_at_hour = created_at_time.hour

    if created_at_hour in created_at_hours:
        created_at_hours[created_at_hour] += 1
        comments_at_hours[created_at_hour] += amount_of_comments
    else:
        created_at_hours[created_at_hour] = 1
        comments_at_hours[created_at_hour] = amount_of_comments

print('Created at hours:', created_at_hours)
print('Comments at hours:', comments_at_hours)

average_comments_at_hours = {}

for hour in comments_at_hours:
    average_comments_at_hours[hour] = comments_at_hours[hour] / created_at_hours[hour]


average_comments_at_hours = {k: v for k, v in sorted(average_comments_at_hours.items(), key=lambda item: item[1])}

print('Average comments at hours:', average_comments_at_hours)

Created at hours: {9: 45, 13: 85, 10: 59, 14: 107, 16: 108, 23: 68, 12: 73, 17: 100, 15: 116, 21: 109, 20: 80, 2: 58, 18: 109, 3: 54, 5: 46, 19: 110, 1: 60, 22: 71, 8: 48, 4: 47, 0: 55, 6: 44, 7: 34, 11: 58}
Comments at hours: {9: 251, 13: 1253, 10: 793, 14: 1416, 16: 1814, 23: 543, 12: 687, 17: 1146, 15: 4477, 21: 1745, 20: 1722, 2: 1381, 18: 1439, 3: 421, 5: 464, 19: 1188, 1: 683, 22: 479, 8: 492, 4: 337, 0: 447, 6: 397, 7: 267, 11: 641}
Average comments at hours: {9: 5.5777777777777775, 22: 6.746478873239437, 4: 7.170212765957447, 3: 7.796296296296297, 7: 7.852941176470588, 23: 7.985294117647059, 0: 8.127272727272727, 6: 9.022727272727273, 12: 9.41095890410959, 5: 10.08695652173913, 8: 10.25, 19: 10.8, 11: 11.051724137931034, 1: 11.383333333333333, 17: 11.46, 18: 13.20183486238532, 14: 13.233644859813085, 10: 13.440677966101696, 13: 14.741176470588234, 21: 16.009174311926607, 16: 16.796296296296298, 20: 21.525, 2: 23.810344827586206, 15: 38.5948275862069}
